# Virtual channel

In [2]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

In [3]:
import LDAQ

In [4]:
# Create 1st serial acquisition source:
acq_arduino = LDAQ.serial_communication.SerialAcquisition(port="COM3", baudrate=250000, 
                                                acquisition_name="arduino",
                                                byte_sequence=(("int16", 2), ),
                                                start_bytes=  b"\xfa\xfb",
                                                end_bytes= b"\n",
                                                sample_rate=500,
                                                channel_names=["channel 1", "channel 2"])

# Here the arduino sends out bytes over serial where a line would look like this:
# b"\xfa\xfb\x00\x01\x00\x02\n

# Explanation:
# b"\xfa\xfb" are the start bytes
# b"\n" is the end byte
# b"\x00\x01" is the first sample (int16)
# b"\x00\x02" is the second sample (int16)

In [6]:
# add virtual channel:
def fun_ratio(ch1, ch2):
    return ch1 / ch2 # ensure that shape is (N, 1) and not (N, )

acq_arduino.add_virtual_channel(virtual_channel_name="ratio channel",
                                source_channels=["channel 1", "channel 2"],
                                function=fun_ratio)

# add virtual channel:
current_sum = 0
def fun_cumulative_sum(ch2):
    global current_sum
    local_sum = np.cumsum(ch2) + current_sum
    local_sum = local_sum.reshape(-1, 1)
    
    if local_sum.shape[0] > 0:
        current_sum = local_sum[-1]
        return  local_sum # ensure that shape is (N, 1) and not (N, )
    else:
        return np.empty((0, 1))

acq_arduino.add_virtual_channel(virtual_channel_name="cumsum channel",
                                source_channels=["channel 2"],
                                function=fun_cumulative_sum)

In [7]:
# Create visualization:
vis = LDAQ.Visualization(refresh_rate=30)
vis.add_lines((0, 0), "arduino", ["channel 1"]) # channel 1
vis.add_lines((0, 1), "arduino", ["channel 2"]) # channel 2
vis.add_lines((1, 0), "arduino", ["ratio channel"]) # virtual channel
vis.add_lines((2, 0), "arduino", ["cumsum channel"]) # virtual channel

vis.config_subplot((0, 0), t_span=0.5)
vis.config_subplot((0, 1), t_span=0.5)
vis.config_subplot((1, 0), t_span=5.0, colspan=2)
vis.config_subplot((2, 0), t_span=5.0, colspan=2)

In [8]:
# create core object and add acquisition sources:
ldaq = LDAQ.Core(acquisitions=[acq_arduino], visualization=vis)
# set trigger:
ldaq.set_trigger(
    source  = "arduino",
    channel = "channel 2",
    level   = 100,
    duration= 60.) # if check event not triggered, measurement will run for 60 seconds

# run acquisition:
ldaq.run()

In [9]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement

{'arduino': {'time': array([0.0000e+00, 2.0000e-03, 4.0000e-03, ..., 1.1756e+01, 1.1758e+01,
         1.1760e+01]),
  'channel_names': ['channel 1',
   'channel 2',
   'ratio channel',
   'cumsum channel'],
  'data': array([[ 5.12000000e+02,  5.76000000e+02,  8.88888889e-01,
           5.78709459e+02],
         [ 5.12000000e+02,  6.39000000e+02,  8.01251956e-01,
           1.21770946e+03],
         [ 5.12000000e+02,  7.00000000e+02,  7.31428571e-01,
           1.91770946e+03],
         ...,
         [-5.12000000e+02,  2.65000000e+02, -1.93207547e+00,
           3.01461971e+06],
         [-5.12000000e+02,  2.11000000e+02, -2.42654028e+00,
           3.01483071e+06],
         [-5.12000000e+02,  1.61000000e+02, -3.18012422e+00,
           3.01499171e+06]]),
  'sample_rate': 500}}